[Home](../index.ipynb) / ADC-Wandler
***
<span style="font-size:20pt;">ADC-Wandler</span>

---
# Lösungsvorschlag

## Aufgabe
* Mit einem Arduino kompatiblen Board soll eine automatische Dimmerschaltung entwickelt werden die obiges Problem behebt:  
  die Beleuchtung soll bei weniger Umgebungslicht automatisch schwächer werden.
* Die Beleuchtung soll durch eine externe blaue LED realisiert werden, der Umgebungshelligkeitssensor mit einem LDR.  
  Die minimale und die maximale Helligkeit sollen jeweils über eine Variable festgelegt werden (sie sollen später mit einem Rotary-Encoder eingestellt werden können).

In [ ]:
#######################################
# Automatic dimmer
#######################################

%serialconnect
# --port=COM3 # for Windows with more than one COM port

#======================================
# Constants:

PIN_ADC = 32
PIN_LED = 14

#--------------------------------------

FREQUENCY = 1000

VALUE_MIN = 65536/2
VALUE_MAX =     0

LED_DUTY_MIN = 20
LED_DUTY_MAX = FREQUENCY


#--------------------------------------

VALUE2DUTY_M = (LED_DUTY_MAX-LED_DUTY_MIN)/(VALUE_MAX-VALUE_MIN)
VALUE2DUTY_T = LED_DUTY_MIN - VALUE2DUTY_M * VALUE_MIN


#======================================
# init ADC and LED

from machine import ADC, PWM, Pin

#adc = ADC(0)       # Croduino
adc = ADC(Pin(PIN_ADC) ) # HelTec, NodeMCU
adc.atten(ADC.ATTN_6DB)


pinLED = PWM( Pin(PIN_LED, Pin.OUT), freq=FREQUENCY, duty=LED_DUTY_MIN)


#======================================
# Loop "forever"


iDutyLast = LED_DUTY_MIN

try:
    while True:
        iValue = adc.read_u16()  # read a raw analog value in the range 0-65535 (= 2^16 - 1)
        #iValue = adc.read()      # Croduino: read a raw analog value in the range 0-511 (= 2^7 - 1)
        
        iDuty = VALUE2DUTY_M * iValue + VALUE2DUTY_T
        
        if   iDuty < LED_DUTY_MIN : iDuty = LED_DUTY_MIN
        elif iDuty > LED_DUTY_MAX : iDuty = LED_DUTY_MAX
        
        iDuty = round( iDuty )

        if iDuty != iDutyLast:
            pinLED.duty(iDuty)
            iDutyLast = iDuty

except KeyboardInterrupt:
    pass
finally:
    pinLED.deinit()

print( "\nDone." )